In [3]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
import pandas as pd
import joblib
import os
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from matplotlib.figure import Figure


# Charger le modèle et le scaler
def load_model_and_scaler():
    model_file = "logistic_model.pkl"
    scaler_file = "scaler.pkl"
    
    if not os.path.exists(model_file) or not os.path.exists(scaler_file):
        messagebox.showerror("Erreur", "Modèle ou scaler introuvable. Veuillez exécuter le 1ᵉʳ notebook pour les générer.")
        root.quit()
    
    model = joblib.load(model_file)
    scaler = joblib.load(scaler_file)
    return model, scaler


# Fonction pour charger un CSV et prédire
def load_csv_and_predict():
    file_path = filedialog.askopenfilename(filetypes=[("Fichiers CSV", "*.csv")])
    if not file_path:
        return
    
    try:
        # Récupérer le séparateur sélectionné
        separator = separator_combobox.get()
        if not separator:
            separator = ";"  # Défaut si aucun choix
        
        # Charger les données CSV
        data = pd.read_csv(file_path, sep=separator)
        required_columns = ['diagonal', 'height_left', 'height_right', 'margin_low', 'margin_up', 'length']
        
        # Vérifier les colonnes requises
        if not all(col in data.columns for col in required_columns):
            raise ValueError(f"Le fichier doit contenir les colonnes suivantes : {required_columns}")
        
        # Standardiser les données et prédire
        X = data[required_columns]
        X_scaled = scaler.transform(X)
        
        # Prédictions
        data['prédiction'] = model.predict(X_scaled)
        data['prédiction'] = data['prédiction'].map({1: "Vrai billet", 0: "Faux billet"})  
        
        # Probabilités de prédiction
        probabilities = model.predict_proba(X_scaled)[:, 1]  # Probabilité de "Vrai billet"
        data['précision'] = (probabilities * 100).round(3)  # Convertir en pourcentage
        
        # Calculer le nombre de vrais et faux billets
        num_genuine = (data['prédiction'] == "Vrai billet").sum()
        num_fake = (data['prédiction'] == "Faux billet").sum()
        
        # Mettre à jour les labels des résultats
        genuine_label.config(text=f"Vrais billets : {num_genuine}", fg="green")
        fake_label.config(text=f"Faux billets : {num_fake}", fg="red")
        
        # Afficher les résultats détaillés et les graphiques dans une nouvelle fenêtre
        display_results_and_charts(data, num_genuine, num_fake)
    
    except Exception as e:
        messagebox.showerror("Erreur", str(e))


# Fonction pour afficher les résultats détaillés et les graphiques
def display_results_and_charts(data, num_genuine, num_fake):
    results_window = tk.Toplevel(root)
    results_window.title("Résultats et Visualisations")
    
    # Frame pour le tableau
    frame_table = ttk.Frame(results_window)
    frame_table.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=10, pady=10)
    
    # Tableau des résultats
    tree = ttk.Treeview(frame_table, columns=list(data.columns), show="headings")
    for col in data.columns:
        tree.heading(col, text=col)
        tree.column(col, anchor=tk.CENTER, width=100)
    
    for _, row in data.iterrows():
        tree.insert("", tk.END, values=list(row))
    
    tree.pack(fill=tk.BOTH, expand=True)

    # Frame pour les graphiques
    frame_charts = ttk.Frame(results_window)
    frame_charts.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=10, pady=10)
    
    # Création de l'histogramme
    fig = Figure(figsize=(5, 4), dpi=100)
    ax = fig.add_subplot(111)
    ax.bar(['Vrais billets', 'Faux billets'], [num_genuine, num_fake], color=['green', 'red'])
    ax.set_title("Distribution des billets")
    ax.set_ylabel("Nombre de billets")
    
    # Ajouter l'histogramme à Tkinter
    canvas = FigureCanvasTkAgg(fig, frame_charts)
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    canvas.draw()

    # Création du camembert
    fig_pie = Figure(figsize=(5, 4), dpi=100)
    ax_pie = fig_pie.add_subplot(111)
    sizes = [num_genuine, num_fake]
    labels = ['Vrais billets', 'Faux billets']
    colors = ['green', 'red']
    explode = (0.1, 0)  # Décalage pour le camembert
    ax_pie.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax_pie.set_title("Proportion des vrais et faux billets")
    
    # Ajouter le camembert à Tkinter
    canvas_pie = FigureCanvasTkAgg(fig_pie, frame_charts)
    canvas_pie.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    canvas_pie.draw()


# Fonction pour exporter les résultats
def export_results(data):
    file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV Files", "*.csv")])
    if file_path:
        data.to_csv(file_path, index=False)
        messagebox.showinfo("Succès", f"Résultats exportés : {file_path}")


# Interface principale
root = tk.Tk()
root.title("Application de prédiction de billets")
root.geometry("900x450")

# Charger le modèle et le scaler
model, scaler = load_model_and_scaler()

# Interface graphique
label = tk.Label(root, text="Application de prédiction de billets", font=("Helvetica", 16))
label.pack(pady=10)

# Menu déroulant pour sélectionner le séparateur
separator_label = tk.Label(root, text="Choisissez le séparateur CSV :", font=("Helvetica", 12))
separator_label.pack(pady=5)

separator_combobox = ttk.Combobox(root, values=[",", ";", "\t"], state="readonly", width=10)
separator_combobox.set(";")  # Valeur par défaut
separator_combobox.pack(pady=5)

# Bouton pour charger un fichier
button = ttk.Button(root, text="Charger un fichier CSV", command=load_csv_and_predict)
button.pack(pady=10)

# Labels pour afficher les vrais et faux billets avec des couleurs distinctes
genuine_label = tk.Label(root, text="Vrais billets : 0", font=("Helvetica", 14), fg="green")
genuine_label.pack(pady=5)

fake_label = tk.Label(root, text="Faux billets : 0", font=("Helvetica", 14), fg="red")
fake_label.pack(pady=5)

root.mainloop()
